# Project : Instagram Data Clustering

# 1. Project Introduce
### 1.1 Purpose : 
문장이 주어지면 주제(Topic)을 알아내는 프로젝트입니다.

그로인한 기대효과로 크게는
1. 사용자가 쓰는 글 내용의 Topic을 알아내고 분석하여 추천 시스템의 기반을 마련하는 것
2. 결정된 Topic이 가진 단어 분포(우리는 Hashtag를 통해서 모델링 했으므로 이 경우는 Hashtag를 의미함)를 이용해서 Hashtag를 생성

이렇게 2 가지가 있습니다.

# 2. How
1. feature vector로는 유저가 각 포스팅에 사용한 Hashtag를 Countvectorizer를 통해서 인코딩한 것을 사용했습니다.
2. Latent Dirichlet Allocation (LDA) algorithm을 사용해서 클러스터링을 진행했고, 그 가장 높은 Topic 결과를 각 문장의 Topic column(Target column)으로 설정하였습니다.
3. 사용자가 쓴 글 내용(Caption)에서 tag는 제외하고(caption_only)을 다시 tf-idf vectorizer로 인코딩하여 featrue vector로 삼고 Machine learning 기법들(Multinomial Naive Bayse, XGBoost 등)을 활용해서 학습시켰습니다.
  
# 2. Data Source
### 2.1 Scrapy :
Scrapy framework를 사용하여 Instagram Hastag로 검색된 결과들을 Crawling 할 수 있는 Crawler를 설계하였습니다.
<img src='./scrapy_screenshot.png'>

> Instagram은 자신들에의해 검증된 어플리케이션 외에는 API를 제공하지 않습니다. 그래서 ```https://www.instagram.com/explore/tags/{tag_name}/?__a=1``` 의 방법을 사용하여 크롤링을 진행하였습니다.
    
# 3. Data Introduce
### 3.1 Data set :
   > - Train Data : 38791 rows, 20 columns
   
### 3.2 Columns : 
- 포스팅 json에서 추출해낸 columns
    - caption : 원본 글 내용
    - comment_cnt : 댓글 개수
    - first_comment : 첫 댓글(본인이 남긴 경우는 보통 Tag인 경우가 대부분)
    - id : 글 ID
    - is_video : 동영상인지 여부
    - likes : 좋아요 개수
    - loc_id : location id
    - loc_lat : 위도
    - loc_lon : 경도
    - owner_id : 유저 ID
    - owner_name : 닉네임
    - shortcode : 글 고유 번호
    - taken_at_timestamp : 저장된 시간
    - video_view_count : 동영상 조회 횟수
    
- caption을 이용해서 만들어낸 columns
    - tags : 사용한 hashtag list
    - tags_cnt : hashtag 개수
    - caption_only : 태그 제외한 글 내용
    - tags_str : hashtag 문자열
    - duplicated_tag : 중복된 태그가 있는지 여부(보통은 두번 쓰지않음)
    
# 4. Algorithms
### 4.1 LDA Topic Modeling
sklearn의 decomposition.LatentDirichletAllocation class를 사용

### 4.2 Multinomial Naive Bayes Classifier
sklearn.naive_bayes.MultinomialNB class를 사용

### 4.3 Hyperparameter Tunning
sklearn.model_selection.GridSearchCV class를 사용

# 5. Conculsion(2019. 1. 21일 기준)
### 5.1 Problems
1. Hashtag의 남용. 사용자들이 사진과 관련없는 내용도 tag에 추가하는 경우가 많았습니다. (#일상#ootd#f4f#선팔맞팔#.. 등등)
    - 해결법 : 
        1. 이 문제의 해결법으로 많이 사용되는 tag들은 제거하고 Topic 모델링을 해서 좀 더 핵심이 되는 tag들에 의존하는 방법을 사용할 수 있습니다.
        2. LDA 모델링 후, 머신러닝을 통해서 caption을 형태소 분석기를 통해서 좀 더 정밀한 전처리를 실시하는 방법이 있습니다.
        
2. 비대칭 데이터.
    - 해결법 : Oversampling이나 Undersampling을 이용합니다.

### 5.2 Result
결과는 현재 이상적이지 않습니다. Accuracy 0.62716

In [1]:
# Import libraries
from IPython.display import display, Markdown
import private_function as pf
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.externals import joblib
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from imblearn.under_sampling import *
from imblearn.over_sampling import *
from imblearn.combine import *

# Pretty display for notebooks
%matplotlib inline

# Ignore the warnings
import warnings
warnings.filterwarnings('ignore')

def get_x_train_and_features_name(df):
    cnt_vectorizer = CountVectorizer(lowercase=False)
    X_train = cnt_vectorizer.fit_transform(df)
#     cnt_feature_names = cnt_vectorizer.get_feature_names()
    return X_train

def run_lda(df, n_topic, max_iter = 100):
    cnt_vectorizer = CountVectorizer(lowercase=False)
    X_train = cnt_vectorizer.fit_transform(df)
    cnt_feature_names = cnt_vectorizer.get_feature_names()

    # hyper parameter
    alpha = 1
    beta = 1

    # train the model
    lda = LatentDirichletAllocation(n_components=n_topic, doc_topic_prior=alpha,\
                                    topic_word_prior=beta, learning_method='online', max_iter=max_iter)

    %time lda.fit_transform(X_train)
    
    return lda, cnt_vectorizer, cnt_feature_names

In [2]:
df_li = pf.getAllDataFrame()
train = pf.makeOneTrainDf(df_li)
train = pf.make_df_i_want(train)
counter = pf.get_counter(train)
counter.most_common()
tag_count = 0
for tup in counter.most_common():
    tag_count += tup[1]
tag_count, len(counter)

['직장인', '럽스타그램', '서울맛집', '먹스타그램', '친스타그램', '여행', '일상', '셀스타그램']

['seoul']

job.csv : 2213

insta_new_5.csv : 2136

insta_new_4.csv : 2148

...

seoul.csv : 880

yongin.csv : 960

gyeonggido.csv : 960

29
Df list length : 29
Df length changes after concat (if 0 means all datas are unique) : 0
Df length changes after concat (if 0 means all datas are unique) : 0
Df length changes after concat (if 0 means all datas are unique) : 1095
Df length changes after concat (if 0 means all datas are unique) : 411
Df length changes after concat (if 0 means all datas are unique) : 54
Df length changes after concat (if 0 means all datas are unique) : 131
Df length changes after concat (if 0 means all datas are unique) : 0
Df length changes after concat (if 0 means all datas are unique) : 35
Df length changes after concat (if 0 means all datas are unique) : 16
Df length changes after concat (if 0 means all datas are unique) : 75
Df length changes after concat (if 0 means all datas are unique) : 11
Df length changes after concat (if 0 means all datas are unique) : 42
Df length changes after concat (if 0 means all datas are unique) : 358
Df length changes after concat (if 0 means all datas are unique) 

(46899, 14241)

In [3]:
df_train = train[train.tags_cnt != 0]

In [4]:
df_train.shape

(38791, 20)

In [5]:
def load_lda_model(n_topic):
    n_topic = str(n_topic)
    lda = joblib.load("./lda_model" + n_topic + ".pkl")
    cnt_feature_name = joblib.load("./lda_model" + n_topic + "_feature_name.pkl")
    return lda, cnt_feature_name

In [6]:
%time lda_model14, cnt_feature_names14 = load_lda_model(14)

CPU times: user 406 ms, sys: 29.2 ms, total: 436 ms
Wall time: 442 ms


## LDA Modeling 후 결론

> Topic 개수는 11-15개까지 바꿔가면서 시도했습니다. 문제는 최적의 Topic 개수를 찾는 것이었는데, Perplexity가 Topic을 늘릴수록 늘어났습니다. 그럼에도 14개를 선택한 것은 직접 눈으로 보고 판단하여 가장 합리적으로 잘 분류됐다고 판단했기 때문입니다. Topic별 Top 20개의 Hashtag를 살펴보면 아래와 같습니다.

### Topic 0: 여행스타그램
여행에미치다 여행스타그램 여행 travel 휴가 korea photography 일본 photo seoul 바다 감성사진 한국 trip 풍경 웨딩 럽스타그램 스냅 예신 결혼

### Topic 1: 대출 광고
신용카드현금화 카드깡 상품권현금화 소액결제 소액결제현금화 모바일문화상품권 굿핀 해피머니 컬쳐랜드 신용카드대출 일수대출 모바 홍대맛집 휴대폰소액결제 차스타그램 핸드폰소액결제 빈티지 골프 상품권 비트코인

### Topic 2: 운동스타그램
다이어트 운동하는여자 다이어터 다이어트식단 용인 diet 다이어트그램 꽃다발 꽃스타그램 식단 헬스 flower 운동 운스타그램 fitness 플로리스트 유지어터 workout 운동하는남자 식단일기

### Topic 3: 신발 광고
축구 헬스 커플신발 푸마 레플 해외직구 아웃도어 커플운동화 나이키에어맥스 신발쇼핑몰 신상신발 등산화 신발도매 명품등산화 유행신발 아디다스신발 GGDB 아식스 아디다스울트라부스트 NEWBALANCE

### Topic 4: Kpop
kpop bts exo 귀여운 blackpink 여자 korea kawaii 아름다운 自撮り boy asmr korean 속초맛집 Asia jisoo kpopl4l tomboy селфи 自分撮り

### Topic 5: 서울 맛집
서울맛집 부산맛집 강남맛집 대구맛집 곱창 홍대맛집 강남역 강남역맛집 대전맛집 강남 맛스타그램 역삼맛집 이태원맛집 역삼동맛집 신논현맛집 대치동맛집 서면맛집 곱창맛집 역삼동 서초맛집

### Topic 6: 럽스타그램
남친이랑 데이트 커플 사랑해 럽스타 커플스타그램 남자친구 영화 연애중 연애 행복 남친 고마워 사랑꾼 ㅋㅋㅋ 행복해 데이트그램 사랑 여자친구 화이팅

### Topic 7: 육아스타그램
럽스타그램 육아 육아스타그램 육아소통 육아맘 맘스타그램 도치맘 일상 젊줌마 딸스타그램 인스타베이비 애스타그램 직장인 아들스타그램 사랑해 아들맘 세젤귀 반려견 워킹맘 멍스타그램

### Topic 8: 토토 등 사이트 및 명품 광고
토토사이트추천 겐조 샤넬 서울맛집 운동하는남자 럽스타 핫플레이스 커플 구찌 에르메스 술스타그램 토토사이트 발렌티노 팔찌 스포츠가족방 사설사이트추천 파워볼가족방 프로토가족방 픽스터가족방 사설사이트

### Topic 9: 폰케이스 광고
선물 직장인 아이폰케이스 냥스타그램 마카롱 취미 고양이 폰케이스 아이폰8 커플케이스 공감 그림 글스타그램 기념일 디저트 일러스트 아이폰7케이스 아이폰x케이스 글귀 귀걸이

### Topic 10: 먹스타그램
먹스타그램 먹방 맛스타그램 맛집 맥주스타그램 food 술스타그램 점심 먹스타 맥주 instafood foodstagram 먹방스타그램 음식 존맛 맛스타 먹부림 푸드스타그램 맛있다 디저트

### Topic 11: 일상적인 해시태그
일상 맞팔 데일리 소통 선팔 셀스타그램 좋아요 셀카 셀피 팔로우 먹스타그램 daily 얼스타그램 일상스타그램 선팔하면맞팔 오오티디 럽스타그램 인친 좋아요반사 ootd

### Topic 12: 제주도 맛집 광고
제주도 제주도맛집 제주맛집 제주 서귀포맛집 제주여행 f4follow 선팔맞팔 제주맛집추천 소통해요 소통하자 제주도그램 제주서귀포맛집 맥주스타그램 제주도여행 제주도흑돼지맛집 제주흑돼지 제주도흑돼지 서귀포흑돼지맛집 서귀포흑돼지

### Topic 13: 거제도 맛집?
거제도맛집 거제맛집 포천여행 짖어야개다 평일 포천카페 햄스터 포천이동갈비 삼척맛집 성남애견미용 근무시간 스킨케어 포천맛집 거제대명리조트맛집 성남애견호텔 골든햄스터 살롱순라 햄스타그램 2018년 삼척여행

### 1, 3, 8, 9, 12는 다 광고성 글이다

In [7]:
def get_topic_using_lda(lda_model, x_train):
    doc_topic_dist_unnormalized = np.matrix(lda_model.transform(x_train))
    # normalize the distribution (only needed if you want to work with the probabilities)
    doc_topic_dist = doc_topic_dist_unnormalized/doc_topic_dist_unnormalized.sum(axis=1)
    return doc_topic_dist.argmax(axis=1)

In [12]:
X_train = df_train[df_train["tags_str"] != ""]["tags_str"].values

In [13]:
x_train = get_x_train_and_features_name(X_train)

In [14]:
y_pred = get_topic_using_lda(lda_model14, x_train)

In [54]:
df_train = train[train["tags_str"] != ""]

In [15]:
df_train["topic_type"] = y_pred

In [16]:
df_train.topic_type.unique()

array([11,  7,  9,  0,  3,  6, 10,  5,  2,  8,  1, 13, 12,  4])

In [65]:
pf.saveDf(df_train, "df_train_14_topic")

In [66]:
df_train.to_pickle("asset/df_train_14_topic.pkl")

In [17]:
tmp_li = [1, 3, 8, 9, 12]
count = 0
for topic in tmp_li:
    length = len(df_train[df_train.topic_type == topic])
    count += length
    print(topic, length)
count

1 577
3 887
8 1453
9 1449
12 531


4897

## 광고에 해당하는 것으로 보이는 데이터가 4897개이다.

In [18]:
df_train["caption_only"] = df_train["caption_only"].apply(lambda a: "" if type(a) == float else a)
df_train["caption_only"] = df_train["caption_only"].apply(lambda a: a.strip())

In [19]:
train_x = df_train[df_train["caption_only"] != ""]

In [20]:
tfidf_max_df_params = np.arange(0.8, 1, 0.05)
# tfidf_min_df_params = np.arange(0, 0.3, 0.05)
tfidf_max_df_params

array([0.8 , 0.85, 0.9 , 0.95])

In [21]:
X_train, X_test, y_train, y_test = train_test_split(train_x.caption_only, train_x.topic_type, test_size=0.1, random_state=1)
len(X_train), len(X_test), len(y_train), len(y_test)

(32796, 3645, 32796, 3645)

In [22]:
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB(alpha=0.01)) 
])

parameters = {
    'tfidf__max_df': tfidf_max_df_params,
#     'tfidf__min_df': tfidf_min_df_params,
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)]
}

grid_search_tune = GridSearchCV(pipeline, parameters, cv=10, n_jobs=3, verbose=3)

# %%time
grid_search_tune.fit(X_train, y_train)

# print("Best parameters set:")
# print(grid_search_tune.best_estimator_.steps)

Fitting 10 folds for each of 12 candidates, totalling 120 fits
[CV] tfidf__max_df=0.8, tfidf__ngram_range=(1, 1) ....................
[CV] tfidf__max_df=0.8, tfidf__ngram_range=(1, 1) ....................
[CV] tfidf__max_df=0.8, tfidf__ngram_range=(1, 1) ....................
[CV]  tfidf__max_df=0.8, tfidf__ngram_range=(1, 1), score=0.604261796042618, total=   1.3s
[CV]  tfidf__max_df=0.8, tfidf__ngram_range=(1, 1), score=0.5921413341455986, total=   1.3s
[CV] tfidf__max_df=0.8, tfidf__ngram_range=(1, 1) ....................
[CV]  tfidf__max_df=0.8, tfidf__ngram_range=(1, 1), score=0.6093845216331505, total=   1.3s
[CV] tfidf__max_df=0.8, tfidf__ngram_range=(1, 1) ....................
[CV] tfidf__max_df=0.8, tfidf__ngram_range=(1, 1) ....................
[CV]  tfidf__max_df=0.8, tfidf__ngram_range=(1, 1), score=0.5973788479122218, total=   1.3s
[CV] tfidf__max_df=0.8, tfidf__ngram_range=(1, 1) ....................
[CV]  tfidf__max_df=0.8, tfidf__ngram_range=(1, 1), score=0.6022554099359

[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed:   39.4s


[CV]  tfidf__max_df=0.8, tfidf__ngram_range=(1, 3), score=0.6187862153095456, total=   5.2s
[CV] tfidf__max_df=0.8, tfidf__ngram_range=(1, 3) ....................
[CV]  tfidf__max_df=0.8, tfidf__ngram_range=(1, 3), score=0.5979255643685174, total=   5.2s
[CV] tfidf__max_df=0.8500000000000001, tfidf__ngram_range=(1, 1) .....
[CV]  tfidf__max_df=0.8500000000000001, tfidf__ngram_range=(1, 1), score=0.604261796042618, total=   1.4s
[CV] tfidf__max_df=0.8500000000000001, tfidf__ngram_range=(1, 1) .....
[CV]  tfidf__max_df=0.8, tfidf__ngram_range=(1, 3), score=0.6058015267175573, total=   5.3s
[CV] tfidf__max_df=0.8500000000000001, tfidf__ngram_range=(1, 1) .....
[CV]  tfidf__max_df=0.8500000000000001, tfidf__ngram_range=(1, 1), score=0.5921413341455986, total=   1.3s
[CV] tfidf__max_df=0.8500000000000001, tfidf__ngram_range=(1, 1) .....
[CV]  tfidf__max_df=0.8500000000000001, tfidf__ngram_range=(1, 1), score=0.6093845216331505, total=   1.4s
[CV] tfidf__max_df=0.8500000000000001, tfidf__ngr

[CV] tfidf__max_df=0.9000000000000001, tfidf__ngram_range=(1, 2) .....
[CV]  tfidf__max_df=0.9000000000000001, tfidf__ngram_range=(1, 2), score=0.6080463273392258, total=   3.4s
[CV] tfidf__max_df=0.9000000000000001, tfidf__ngram_range=(1, 2) .....
[CV]  tfidf__max_df=0.9000000000000001, tfidf__ngram_range=(1, 2), score=0.6083511124657117, total=   3.4s
[CV] tfidf__max_df=0.9000000000000001, tfidf__ngram_range=(1, 2) .....
[CV]  tfidf__max_df=0.9000000000000001, tfidf__ngram_range=(1, 2), score=0.6048780487804878, total=   3.4s
[CV] tfidf__max_df=0.9000000000000001, tfidf__ngram_range=(1, 2) .....
[CV]  tfidf__max_df=0.9000000000000001, tfidf__ngram_range=(1, 2), score=0.6181762732540409, total=   3.5s
[CV] tfidf__max_df=0.9000000000000001, tfidf__ngram_range=(1, 2) .....
[CV]  tfidf__max_df=0.9000000000000001, tfidf__ngram_range=(1, 2), score=0.5933496034167175, total=   3.5s
[CV] tfidf__max_df=0.9000000000000001, tfidf__ngram_range=(1, 3) .....
[CV]  tfidf__max_df=0.9000000000000001,

[Parallel(n_jobs=3)]: Done 120 out of 120 | elapsed:  3.1min finished


GridSearchCV(cv=10, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...ue,
        vocabulary=None)), ('clf', MultinomialNB(alpha=0.01, class_prior=None, fit_prior=True))]),
       fit_params=None, iid=True, n_jobs=3,
       param_grid={'tfidf__max_df': array([0.8 , 0.85, 0.9 , 0.95]), 'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [23]:
grid_search_tune.best_params_, grid_search_tune.best_score_

({'tfidf__max_df': 0.8, 'tfidf__ngram_range': (1, 3)}, 0.6118429076716673)

## Best score를 낸 parameter max_df : 0.8, ngram_range : (1, 3)

In [85]:
grid_search_tune.best_score_

0.6118429076716673

In [24]:
clf = Pipeline([ 
    ('vect', TfidfVectorizer(ngram_range=(1, 3))), 
    ('clf', MultinomialNB(alpha=0.01)) 
])

In [25]:
%%time
model_all_data = clf.fit(X_train, y_train)

CPU times: user 4.75 s, sys: 275 ms, total: 5.02 s
Wall time: 5.02 s


In [26]:
y_pred = model_all_data.predict(X_test)

In [27]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.55      0.36      0.44       198
          1       0.96      0.72      0.82        67
          2       0.67      0.22      0.33        55
          3       0.40      0.08      0.13        75
          4       0.85      0.39      0.54        28
          5       0.78      0.69      0.73       143
          6       0.33      0.12      0.18       174
          7       0.53      0.45      0.49       343
          8       0.92      0.94      0.93       131
          9       0.76      0.42      0.55       153
         10       0.50      0.58      0.54       573
         11       0.65      0.80      0.72      1632
         12       0.92      0.59      0.72        58
         13       0.89      0.53      0.67        15

avg / total       0.62      0.63      0.61      3645



### Handle imbalance problem

In [2]:
def handle_imbalance(method, x_train, y_train, x_test, y_test, rs=0, use_exact_model = []):
    method_idx = 0
    if method == "under":
        method_idx = 0
    elif method == "over":
        method_idx = 1
    elif method == "combine":
        method_idx = 2
    else:
        print("Error occured.")
        return
    multimodel = MultinomialNB(alpha=0.01)
    tf_vec = TfidfVectorizer(ngram_range=(1, 3))
    x_sample = tf_vec.fit_transform(x_train)
    models = [
        [
            RandomUnderSampler(random_state=rs),
            TomekLinks(random_state=rs),
            CondensedNearestNeighbour(random_state=rs),
            OneSidedSelection(random_state=rs),
            EditedNearestNeighbours(random_state=rs),
            NeighbourhoodCleaningRule(random_state=rs)
        ],
        [
            RandomOverSampler(random_state=rs),
            ADASYN(random_state=rs),
            SMOTE(random_state=rs)
        ],
        [
            SMOTEENN(random_state=rs),
            SMOTETomek(random_state=rs)
        ]
    ]
    
    if len(use_exact_model) != 0:
        method_idx = use_exact_model[0]
        model_idx = use_exact_model[1]
        x_random, y_random = models[method_idx][model_idx].fit_sample(x_sample, y_train)
        multimodel.fit(x_random, y_random)
        X_test = tf_vec.transform(x_test)
        y_pred = multimodel.predict(X_test)
        print(str(idx) + " 번째 모델 Accuracy 결과 : ")
        print(accuracy_score(y_test, y_pred))
        return multimodel
        
    print(method + " sampling started.")
    print("Total sampler length : " + str(len(models[method_idx])))
    for idx, model in enumerate(models[method_idx]):
        x_random, y_random = model.fit_sample(x_sample, y_train)
        multimodel.fit(x_random, y_random)
        X_test = tf_vec.transform(x_test)
        y_pred = multimodel.predict(X_test)
        print(str(idx) + " 번째 모델 Accuracy 결과 : ")
        print(accuracy_score(y_test, y_pred))

# 0.62716으로 TomekLinks를 사용하면 성능이 조금 더 좋아짐

In [109]:
handle_imbalance("under", X_train, y_train, X_test, y_test)

under sampling started.
0 번째 모델 Accuracy 결과 : 
0.3574759945130315
1 번째 모델 Accuracy 결과 : 
0.6271604938271605
2 번째 모델 Accuracy 결과 : 
0.17366255144032922


/usr/local/lib/python3.6/site-packages/imblearn/under_sampling/prototype_selection/one_sided_selection.py:197: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  idx_maj_extracted = np.delete(idx_maj, idx_maj_sample, axis=0)
/usr/local/lib/python3.6/site-packages/imblearn/under_sampling/prototype_selection/one_sided_selection.py:197: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  idx_maj_extracted = np.delete(idx_maj, idx_maj_sample, axis=0)
/usr/local/lib/python3.6/site-packages/imblearn/under_sampling/prototype_selection/one_sided_selection.py:197: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.
  idx_maj_extracted = np.delete(idx_maj, idx_maj_sample, axis=0)
/usr/local/lib/python3.6/site-packages/imblearn/under_sampling/prototype_selection/one_sided_selection.py:197: D

3 번째 모델 Accuracy 결과 : 
0.6183813443072702
4 번째 모델 Accuracy 결과 : 
0.5432098765432098
5 번째 모델 Accuracy 결과 : 
0.574485596707819


## Over sampler 로는 개선된 점이 없다.

In [110]:
handle_imbalance("over", X_train, y_train, X_test, y_test)

over sampling started.
0 번째 모델 Accuracy 결과 : 
0.5097393689986283
1 번째 모델 Accuracy 결과 : 
0.5026063100137175
2 번째 모델 Accuracy 결과 : 
0.5111111111111111


## Over Sampling을 통해서 오히려 성적이 떨어졌고, Combine Sampler들은 현재 용량부족으로 결과를 내지 못하여 일단 보류했다.

In [ ]:
handle_imbalance("combine", X_train, y_train, X_test, y_test)

combine sampling started.
Total sampler length : 2


In [ ]:
mb = handle_imbalance("under", X_train, y_train, X_test, y_test, [0, 1])